In [15]:
# Mount google drive
# ----------------------
from google.colab import drive
drive.mount('/content/drive')
location = 'drive/My Drive/Colab/titanic/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import xgboost as xgb

# Load sklearn utilities
# ----------------------
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, roc_curve, brier_score_loss, mean_squared_error, r2_score

from sklearn.calibration import calibration_curve
from sklearn.preprocessing import OneHotEncoder
from datetime import datetime

# Load classifiers
# ----------------
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.linear_model import Perceptron
from sklearn import svm


In [0]:
# Read Data file
# ----------------------
train = pd.read_csv(location + 'train.csv')
test = pd.read_csv(location + 'test.csv')

In [18]:
# Feature engineering
# ----------------------
X = train[['Pclass', 'Sex', 'Age', 'SibSp',
       'Parch', 'Fare', 'Embarked',
       'Cabin',
       ]]
Y = list(train['Survived'])
X_result = test[['Pclass', 'Sex', 'Age', 'SibSp',
       'Parch', 'Fare', 'Embarked',
       'Cabin',
       ]]
result_id = test['PassengerId']

X = X.fillna(0)
X_result = X_result.fillna(0)
X['Age'][X['Age'] == ''] = X['Age'].mean
X_result['Age'][X_result['Age'] == ''] = X['Age'].mean
X['Cabin'][X['Cabin'] != ''] = 1
X['Cabin'][X['Cabin'] == ''] = 0
X_result['Cabin'][X_result['Cabin'] != ''] = 1
X_result['Cabin'][X_result['Cabin'] == ''] = 0


lenX = len(X)
lenX_result = len(X_result)
temp = pd.get_dummies(pd.concat([X,X_result]))

# scaler = preprocessing.MinMaxScaler()
# temp = scaler.fit_transform(temp)

X = temp[:len(X)]
X_result = temp[len(X):]

# Train/test split
# ----------------------

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

/usr/local/lib/python3.6/dist-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a 

In [0]:
# Data exploration
# ----------------------

# for var in X_train:
#   print(var, set(X_train[var]))
#   try:
#     plt.hist(X_train[var])
#     plt.show()
#   except:
#     print('cannot plot ' + var)


In [0]:
# Fit the model
# ----------------------
def fit_classification(model,cv_parameters = {}):
  cv_model = GridSearchCV(model, cv_parameters)
  cv_model.fit(X_train, Y_train)
  model = cv_model.best_estimator_
  print(model)
  Y_pred = model.predict(X_train)
  Y_pred_test = model.predict(X_test)
  print('Train roc:',roc_auc_score(Y_train, Y_pred))
  print('Test roc:',roc_auc_score(Y_test, Y_pred_test))
  return model
        

In [0]:
# Generate output file
# ----------------------
def write_output(model):
  Y_result = model.predict(X_result)
  output = 'PassengerId,Survived\n'
  for i in range(len(Y_result)):
    output += str(result_id[i]) + ',' +  str(Y_result[i]) + '\n'
  
  f = open(location+(str(datetime.now()) + '.csv'), 'w')
  f.write(output)
  f.close()
    


In [21]:
## Train and test a naive bayes classifier
gnb = fit_classification(GaussianNB(),{})

GaussianNB(priors=None, var_smoothing=1e-09)
Train roc: 0.6847407419839384
Test roc: 0.6887438614629103


In [30]:
## Train and test a perceptron classifier
pct = fit_classification(Perceptron(),{})

Perceptron(alpha=0.0001, class_weight=None, early_stopping=False, eta0=1.0,
           fit_intercept=True, max_iter=1000, n_iter_no_change=5, n_jobs=None,
           penalty=None, random_state=0, shuffle=True, tol=0.001,
           validation_fraction=0.1, verbose=0, warm_start=False)
Train roc: 0.5
Test roc: 0.5


In [37]:
## Train and test a support vector machine classifier
sv = fit_classification(svm.SVC(),{'C':[10000,100,1,0.01]})

SVC(C=10000, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
Train roc: 0.8168410717732258
Test roc: 0.76699405531145


In [0]:
## Train and test a support vector machine classifier
sv = fit_classification(svm.SVC(),{'C':[10000,100,1,0.01]})

In [22]:
## Train and test a l_1 regularized logistic regression classifier
l2_logistic = fit_classification(LogisticRegression(penalty='l2'), {'C':[0.01,0.1,1]})

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
Train roc: 0.7762132133896129
Test roc: 0.7832773326440942


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


In [23]:
## Train and test a decision tree classifier
decision_tree = fit_classification(DecisionTreeClassifier(), {'max_depth':[5,10,None],\
            'max_features':['sqrt','log2',None]
            })

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=5, max_features='log2', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')
Train roc: 0.7901558317319392
Test roc: 0.7785603515120186


In [24]:
## Train and test a multi-layer perceptron classifier
mlp = fit_classification(MLPClassifier(), {
            'hidden_layer_sizes':[5,10,20,50],       
            })

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=50, learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)
Train roc: 0.789488700730907
Test roc: 0.7759757043163609


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [25]:
## Train and test a random forest classifier
rf = fit_classification(RandomForestClassifier(),{
                'max_depth':[2,5,10],\
                 'n_estimators':[20,50,100],\
                 'max_features':[3,5],\
                })


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=10, max_features=5,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)
Train roc: 0.9266449604336771
Test roc: 0.8479581287154303


In [26]:
## Train and test a boosted tree classifier
xgboost = fit_classification(xgb.XGBClassifier(),{
     "eta"    : [0.05, 0.20,0.30 ] ,
     "max_depth"        : [ 3,  6,15],
     "min_child_weight" : [ 1, 4, 7 ],
     "gamma"            : [ 0.0, 0.1,0.4 ],
     "colsample_bytree" : [ 0.3, 0.7 ]
     })


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.3, eta=0.05, gamma=0.1,
              learning_rate=0.1, max_delta_step=0, max_depth=15,
              min_child_weight=4, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)
Train roc: 0.8598605318586522
Test roc: 0.8274101835099509


In [0]:
write_output(xgboost)
write_output(rf)